# Experimentation: Segmenation
---

In [1]:
import os

def delete_license_files(root_dir):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == "LICENSE":
                file_path = os.path.join(dirpath, filename)
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")

# Specify the root folder here
root_folder = '../data'

delete_license_files(root_folder)

In [2]:
# Import libraries
import sys
sys.path.append("..")

from monai.config import print_config
from monai.networks.nets import SwinUNETR
from src.preprocessing import get_transforms, get_datasets, get_dataloaders

import torch

# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# Get transforms
transform = get_transforms()

# Get datasets
train_ds, val_ds = get_datasets(root_dir="../data", collection="HCC-TACE-Seg", transform=transform, download=False, val_frac=0.2)

# Get dataloaders
train_loader, val_loader = get_dataloaders(train_ds, val_ds, batch_size=1)

train_dl =train_ds.datalist
val_dl = val_ds.datalist


# Check length of datasets and dataloaders
print(train_ds.get_indices(), val_ds.get_indices())
print(len(train_loader), len(val_loader))

[ 43  68   3  74  82  75   6  86  84  79  63  97  90  71  53  27  18  59
  89  60  93 103   1  76  42  41   4  15  17  40  38   5  95  54   0  34
  28  55  50  11  35  23  52  10  31  78  57  80  91  32  94  14  73  19
  29  49 101 102  69  20  98  72  77  25  37  81  46  39  65  58  12  92
  88  70  87  36  21  83   9 100  67  64  47  44] [26 61  2 62 85 48 16 99 56 45 66 96 51 13  7 30 22 24 33  8]
84 20


In [4]:
import csv

def datalist_csv(data):

    # Specify the field names
    fieldnames = data[0].keys()

    # Specify the name of the CSV file
    csv_file = "data.csv"

    # Write data to CSV file
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        # Write header
        writer.writeheader()
        
        # Write rows
        for row in data:
            writer.writerow(row)

    print("CSV file created successfully.")


In [5]:
datalist_csv(train_dl)

CSV file created successfully.


In [6]:
# Create a Swin-UNet model
model = SwinUNETR(img_size=(64, 64, 64), in_channels=1, out_channels=5, use_v2 = True, spatial_dims=3)

c:\Users\Sven\anaconda3\envs\torch\Lib\site-packages\monai\utils\deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)


In [7]:
# Set the model to use the GPU
model = model.to(device)

# Set the hyperparameters for training
max_epochs = 10
learning_rate = 1e-3
weight_decay = 1e-5
val_interval = 1

# Set the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay=weight_decay)
loss_function = torch.nn.CrossEntropyLoss()
# Create the training loop

# Set the model to training mode
model.train()

# Iterate over the training data for the specified number of epochs
for epoch in range(max_epochs):
    for batch in train_loader:
        # Extract the image and segmentation from the batch
        image, seg = batch["image"].to(device), batch["seg"].to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(image)
        
        # Compute the loss
        loss = loss_function(output, torch.argmax(seg, dim=1))
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
    # Print the loss for each epoch
    print(f"Epoch [{epoch+1}/{max_epochs}], Loss: {loss.item():.4f}")
    
    # Set the model to evaluation mode
    model.eval()
    
    # Compute the validation loss
    with torch.no_grad():
        for batch in val_loader:
            image, seg = batch["image"].to(device), batch["seg"].to(device)
            output = model(image)
            loss = loss_function(output, torch.argmax(seg, dim=1))
        print(f"Validation Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 0.3004
Validation Loss: 0.4089
Epoch [2/10], Loss: 0.3349
Validation Loss: 0.3094
Epoch [3/10], Loss: 0.1651
Validation Loss: 0.3312
Epoch [4/10], Loss: 0.1194
Validation Loss: 0.3132
Epoch [5/10], Loss: 0.1033
Validation Loss: 0.3225


KeyboardInterrupt: 

In [ ]:
# Save the model
torch.save(model.state_dict(), "../models/swin_unetr.pth")

In [ ]:

# TODO: Saving a CT volume tensor to image format (DICOM / NIFTI)

# Load the model
model.load_state_dict(torch.load("../models/swin_unetr.pth"))

# Set the model to evaluation mode
model.eval()

# Sample a batch of data from the test dataloader to make predictions
batch = next(iter(val_loader))

# Extract the image and segmentation from the batch
image, seg = batch["image"].to(device), batch["seg"].to(device)

# Make predictions on the image
output = model(image)

# Get the predicted segmentation class for each pixel
pred_seg = output


# Visualize the image, ground truth segmentation, and predicted segmentation
import matplotlib.pyplot as plt
import numpy as np

slice_idx = 40

pred_seg = torch.argmax(pred_seg, dim=1).unsqueeze(1)

# Convert the image, ground truth segmentation, and predicted segmentation to NumPy arrays
image = image.cpu().detach().numpy()
seg = seg.cpu().detach().numpy()
pred_seg = pred_seg.cpu().detach().numpy()

# Get the first image, ground truth segmentation, and predicted segmentation from the batch
image = image[0, 0, :, :, slice_idx]
seg = seg[0, 0, :, :, slice_idx]
pred_seg = pred_seg[0, 0, :, :, slice_idx]

# Plot the image, ground truth segmentation, and predicted segmentation
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(image, cmap="gray")
axes[0].set_title("Image")
axes[1].imshow(seg, cmap="gray")
axes[1].set_title("Ground Truth Segmentation")
axes[2].imshow(pred_seg, cmap="tab10")
axes[2].set_title("Predicted Segmentation")
plt.show()

